## Решение практической задачи 1

### Задача: ранжирование потенциальных кандидатов по профессиональным обязанностям

> Решение практической задачи выполняется в два этапа. На первом этапе необходимо использовать библиотеку OCEAN-AI для получения гипотез предсказаний (оценок персональных качеств личности человека). На втором этапе следует использовать методы _candidate_ranking и _priority_skill_calculation из библиотеки OCEAN-AI для решения представленной практической задачи. Примеры результатов работы и реализации представлены ниже.

> Таким образом, библиотека OCEAN-AI предоставляет инструменты для анализа персональных качеств личности кандидатов и их соответствия требованиям должности, что может значительно улучшить процесс подбора персонала и помочь в принятии более объективных и систематизированных решений при ранжировании кандидатов.

<center>
    <img src="https://raw.githubusercontent.com/aimclub/OCEANAI/main/docs/source/user_guide/notebooks/images/Step_1_get_scores_en.svg" width="100%"/>
</center>

<center>
    <img src="https://raw.githubusercontent.com/aimclub/OCEANAI/main/docs/source/user_guide/notebooks/images/Step_2_task_1_en.svg" width="100%"/>
</center>

<hr>

In [1]:
import os  # Взаимодействие с файловой системой
import sys # Доступ к некоторым переменным и функциям Python

PATH_TO_SOURCE = os.path.abspath(os.path.dirname(globals()['_dh'][0]))
PATH_TO_ROOT = os.path.join(PATH_TO_SOURCE, '..', '..', '..')

sys.path.insert(0, os.path.abspath(PATH_TO_ROOT))

### `FI V2`

In [2]:
# Импорт необходимых инструментов
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

corpus = 'fi'

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url, force_reload = False)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang='en')
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['fe']['googledisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url, force_reload = False)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_bert_model = _b5.setup_bert_encoder(force_reload = False)
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['googledisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['text'][corpus]['nn']['googledisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url, force_reload = False)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['googledisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url, force_reload = False)

PATH_TO_DIR = './video_FI/'
PATH_SAVE_VIDEO = './video_FI/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса First Impression V2
# URL: https://chalearnlap.cvc.uab.cat/dataset/24/description/
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '429713680?token=FqHdMLSSh7zYSZt&filename=_plk5k7PBEg.003.mp4',
    '429713681?token=Hz9b4lQkrLfic33&filename=be0DQawtVkE.002.mp4',
    '429713683?token=EgUXS9Xs8xHm5gz&filename=2d6btbaNdfo.000.mp4',
    '429713684?token=1U26753kmPYdIgt&filename=300gK3CnzW0.003.mp4',
    '429713685?token=LyigAWLTzDNwKJO&filename=300gK3CnzW0.001.mp4',
    '429713686?token=EpfRbCKHyuc4HPu&filename=cLaZxEf1nE4.004.mp4',
    '429713687?token=FNTkwqBr4jOS95l&filename=g24JGYuT74A.004.mp4',
    '429713688?token=qDT95nz7hfm2Nki&filename=JZNMxa3OKHY.000.mp4',
    '429713689?token=noLguEGXDpbcKhg&filename=nvlqJbHk_Lc.003.mp4',
    '429713679?token=9L7RQ0hgdJlcek6&filename=4vdJGgZpj4k.003.mp4'
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mp4'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_[corpus]['googledisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = 'en')

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:13:14</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:13:15</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... GitHub\OCEANAI\docs\source\user_guide\notebooks\video_FI\test\_plk5k7PBEg.003.mp4 ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
Person ID,,,,,,
1,2d6btbaNdfo.000.mp4,0.618917,0.660694,0.477656,0.654437,0.601256
2,300gK3CnzW0.001.mp4,0.461732,0.413451,0.415706,0.498301,0.431224
3,300gK3CnzW0.003.mp4,0.468002,0.448618,0.371742,0.509602,0.453739
4,4vdJGgZpj4k.003.mp4,0.585348,0.616446,0.49443,0.605614,0.587017
5,be0DQawtVkE.002.mp4,0.680991,0.56602,0.553915,0.646545,0.64246
6,cLaZxEf1nE4.004.mp4,0.66342,0.551018,0.557912,0.585238,0.587174
7,g24JGYuT74A.004.mp4,0.590237,0.399273,0.409554,0.531861,0.507134
8,JZNMxa3OKHY.000.mp4,0.60577,0.523617,0.531137,0.594406,0.57984
9,nvlqJbHk_Lc.003.mp4,0.511002,0.464702,0.390882,0.443663,0.438811


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:13:15</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.0735,0.0631,0.0914,0.0706,0.0691,0.0735
Accuracy,0.9265,0.9369,0.9086,0.9294,0.9309,0.9265


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:13:15</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.0735</span>, средняя точность: <span style="color:#1776D2">0.9265</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 34.472 сек. ---**</span>

True

<hr>

Для выполнения ранжирования кандидатов необходимо знать весовые коэффициенты, определяющие приоритетность персональных качеств личности в зависимости от профессии.

Предлагаются весовые коэффициенты для 5 профессий, вычисленные на основе научных статей:

1) Sajjad H. et al. Personality and Career Choices // African Journal of Business Management. - 2012. – Vol. 6 (6) – pp. 2255-2260.
2) Alkhelil A. H. The Relationship between Personality Traits and Career Choice: A Case Study of Secondary School Students // International Journal of Academic Research in Progressive Education and Development. – 2016. – Vol. 5(2). – pp. 2226-6348.
3) De Jong N. et al. Personality Traits and Career Role Enactment: Career Role Preferences as a Mediator // Frontiers in Psychology. – 2019. – Vol. 10. – pp. 1720.

Пользователь может установить свои весовые коэффициенты; сумма весов должна быть равна 100.

In [3]:
# Загрузка датафрейма с весовыми коэффициентами
url = 'https://download.sberdisk.ru/download/file/478675798?token=fF5fNZVpthQlEV0&filename=traits_priority_for_professions.csv'
traits_priority_for_professions = pd.read_csv(url)

traits_priority_for_professions.index.name = 'ID'
traits_priority_for_professions.index += 1
traits_priority_for_professions.index = traits_priority_for_professions.index.map(str)

traits_priority_for_professions

,Profession,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,Managers/executives,15,35,15,30,5
2,Entrepreneurship,30,30,5,5,30
3,Social/Non profit making professions,5,5,35,35,20
4,Public sector professions,15,50,15,15,5
5,"Scientists/researchers, and engineers",50,15,5,15,15


#### Ранжирование кандидатов на должность инженера-проектировщика

In [4]:
weights = traits_priority_for_professions.iloc[4].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'engineer_candidate_ranking_fi_en', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
5,be0DQawtVkE.002.mp4,0.681,0.566,0.554,0.647,0.642,64.644
10,_plk5k7PBEg.003.mp4,0.648,0.610,0.525,0.614,0.606,62.472
1,2d6btbaNdfo.000.mp4,0.619,0.661,0.478,0.654,0.601,62.080
6,cLaZxEf1nE4.004.mp4,0.663,0.551,0.558,0.585,0.587,61.812
4,4vdJGgZpj4k.003.mp4,0.585,0.616,0.494,0.606,0.587,58.876
8,JZNMxa3OKHY.000.mp4,0.606,0.524,0.531,0.594,0.580,58.412
7,g24JGYuT74A.004.mp4,0.590,0.399,0.410,0.532,0.507,53.134
9,nvlqJbHk_Lc.003.mp4,0.511,0.465,0.391,0.444,0.439,47.712
3,300gK3CnzW0.003.mp4,0.468,0.449,0.372,0.510,0.454,46.438


#### Ранжирование кандидатов на должность менеджера

In [5]:
weights = traits_priority_for_professions.iloc[0].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'executive_candidate_ranking_fi_en', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
1,2d6btbaNdfo.000.mp4,0.619,0.661,0.478,0.654,0.601,62.212
5,be0DQawtVkE.002.mp4,0.681,0.566,0.554,0.647,0.642,60.943
10,_plk5k7PBEg.003.mp4,0.648,0.610,0.525,0.614,0.606,60.412
4,4vdJGgZpj4k.003.mp4,0.585,0.616,0.494,0.606,0.587,58.876
6,cLaZxEf1nE4.004.mp4,0.663,0.551,0.558,0.585,0.587,58.099
8,JZNMxa3OKHY.000.mp4,0.606,0.524,0.531,0.594,0.580,56.112
7,g24JGYuT74A.004.mp4,0.590,0.399,0.410,0.532,0.507,47.463
3,300gK3CnzW0.003.mp4,0.468,0.449,0.372,0.510,0.454,45.855
9,nvlqJbHk_Lc.003.mp4,0.511,0.465,0.391,0.444,0.439,45.297


<hr>

Для ранжирования кандидатов по профессиональным навыкам необходимо задать по два коэффициента корреляции для каждого персонального качества личности человека и навыка, а также порога полярности качеств. Эти коэффициенты должны показывать, как измениться оценка качества человека если она больше или меньше заданного порога полярности качеств. 

В качестве примера предлагается использование коэффициентов корреляции между двумя людьми в четырьмя профессиональными навыками, представленных в статье:

1) Wehner C., de Grip A., Pfeifer H. Do recruiters select workers with different personality traits for different tasks? A discrete choice experiment // Labour Economics. - 2022. - vol. 78. - pp. 102186.

Представлены 4 профессиональных навыка: 

1) Analytical (Аналитические навыки). Умение эффективно решать новые задачи, требующие глубокого анализа. 
2) Interactive (Навыки межличностного общения). Умение убеждать и достигать компромиссов с заказчиками и коллегами.
4) Routine (Способность выполнять рутинную работу). Умение эффективно управлять рутинными задачами, соблюдая точность и внимание к деталям.
5) Non-Routine (Способность выполнять нестандартную работу). Умение реагировать и решать проблемы, не имеющие установленного порядка, проявляя адаптивность и креативные навыки в решении задач.

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим профессиональным навыкам.

#### Ранжирование кандидатов по профессиональным навыкам

In [6]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478678231?token=0qiZwliLtHWWYMv&filename=professional_skills.csv'
df_professional_skills = pd.read_csv(url)

df_professional_skills.index.name = 'ID'
df_professional_skills.index += 1
df_professional_skills.index = df_professional_skills.index.map(str)

df_professional_skills

,Trait,Score_level,Analytical,Interactive,Routine,Non-Routine
ID,,,,,,
1,Openness,high,0.082,0.348,0.571,0.510
2,Openness,low,0.196,0.152,0.148,0.218
3,Conscientiousness,high,0.994,1.333,1.507,1.258
4,Conscientiousness,low,0.241,0.188,0.191,0.267
5,Extraversion,high,0.169,-0.060,0.258,0.017
6,Extraversion,low,0.181,0.135,0.130,0.194
7,Agreeableness,high,1.239,0.964,1.400,1.191
8,Agreeableness,low,0.226,0.180,0.189,0.259
9,Non-Neuroticism,high,0.636,0.777,0.876,0.729


In [7]:
_b5._priority_skill_calculation(
    correlation_coefficients = df_professional_skills,
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_skill_, name = 'skill_candidate_ranking_fi_en', out = True)

# Опционно
df = _b5.df_files_priority_skill_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Analytical,Interactive,Routine,Non-Routine
Person ID,,,,,,,,,,
1,2d6btbaNdfo.000.mp4,0.619,0.661,0.478,0.654,0.601,0.397,0.452,0.571,0.491
5,be0DQawtVkE.002.mp4,0.681,0.566,0.554,0.647,0.642,0.384,0.416,0.571,0.461
10,_plk5k7PBEg.003.mp4,0.648,0.610,0.525,0.614,0.606,0.379,0.414,0.563,0.456
4,4vdJGgZpj4k.003.mp4,0.585,0.616,0.494,0.606,0.587,0.375,0.426,0.538,0.464
6,cLaZxEf1nE4.004.mp4,0.663,0.551,0.558,0.585,0.587,0.359,0.390,0.537,0.433
8,JZNMxa3OKHY.000.mp4,0.606,0.524,0.531,0.594,0.580,0.353,0.380,0.522,0.421
7,g24JGYuT74A.004.mp4,0.590,0.399,0.410,0.532,0.507,0.240,0.249,0.331,0.298
3,300gK3CnzW0.003.mp4,0.468,0.449,0.372,0.510,0.454,0.198,0.154,0.198,0.202
2,300gK3CnzW0.001.mp4,0.462,0.413,0.416,0.498,0.431,0.093,0.072,0.073,0.105


<hr>

Для ранжирования кандидатов по одному из шестнадцати типов личности MBTI необходимо задать матрицу корреляции между персональными качествами личности человека и четырьмя диспозициями MBTI, установить порог полярности качеств и указать целевой тип личности MBTI.

В качестве примера предлагается использование коэффициентов корреляции, представленных в статье [1]. Описание типов личности MBTI и соотвествующие им успешные профессии представлены в статье [2].

1) Furnham A. The big five facets and the MBTI: The relationship between the 30 NEO-PI (R) Facets and the four Myers-Briggs Type Indicator (MBTI) scores // Psychology. - 2022. vol. 13(10). - pp. 1504-1516.
2) Tieger P.D., Barron B., Tieger K. Do what you are: Discover the perfect career for you through the secrets of personality type // Hachette UK. - 2024.

##### Типы личности MBTI основаны на четырех измерениях личности:

<div style="text-align: center;">
  <table style="width:100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Описание измерения</th>
      <th>Измерение</th>
    </tr>
    <tr>
      <td>Как мы взаимодействуем с миром и куда направляем свою энергию</td>
      <td>(E) Экстраверсия - Интроверсия (I)</td>
    </tr>
    <tr>
      <td>Вид информации, которую мы естественным образом замечаем</td>
      <td>(S) Сенсорика - Интуиция (N)</td>
    </tr>
    <tr>
      <td>Как мы принимаем решения</td>
      <td>(T) Логика - Чувства (F)</td>
    </tr>
    <tr>
      <td>Предпочитаем ли мы жить более структурированно (принимая решения) или более спонтанно (принимая информацию)</td>
      <td>(J) Оценка - Восприятие (P)</td>
    </tr>
  </table>
</div>

##### Типы личности MBTI и соотвествующие им успешные профессии:

<div style="text-align: center;">
  <table style="width: 100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Тип личности</th>
      <th>Описание</th>
      <th>Успешные профессии</th>
    </tr>
    <tr>
      <td>ISTJ</td>
      <td>Этот человек отличается ответственностью, строгостью и педантичностью. Он опирается на объективные факты и склонен к аналитическому мышлению. Приступает к задаче только тогда, когда уверен в своих возможностях и успехе</td>
      <td>Инспектор: бухгалтер, аудитор, бюджетный аналитик, финансовый менеджер, разработчик, системный аналитик, библиотекарь и т. д.</td>
    </tr>
    <tr>
      <td>ISFJ</td>
      <td>Этот человек склонен к самоанализу и анализу окружающих, легко распознает фальшь и предпочитает сохранять психологическую дистанцию. Он исполнителен, внимателен и готов помогать другим. Его силы и энергия исходят из внутренних ресурсов, и он всегда полагается на собственный опыт</td>
      <td>Защитник: медсестра, врач, ветеринар или ветеринарный ассистент, социальный работник, сельскохозяйственный или пищевой ученый, секретарь, водитель и т. д.</td>
    </tr>
    <tr>
      <td>INFJ</td>
      <td>О таких людях говорят: «ему можно доверять». Он отличается высокой чувствительностью, уделяет большое внимание межличностным отношениям, умеет давать ценные советы и помогает раскрывать потенциал других. Развитая интуиция не только генерирует множество идей, но и способствует самоорганизации</td>
      <td>Советник: психолог, специалист по управлению персоналом, офис-менеджер, специалист по обучению, графический дизайнер и т. д.</td>
    </tr>
    <tr>
      <td>INTJ</td>
      <td>Этот человек умеет выделять главное, говорит четко и по существу, придерживается практического подхода. Он стремится постоянно улучшать свою работу и всегда ищет способы сделать задачу еще лучше. Пустые разговоры ему не по душе, поэтому он избегает больших шумных компаний и с трудом заводит новые знакомства</td>
      <td>Мастермайнд: аниматор, архитектор, копирайтер, фотограф, тележурналист, видеомонтажер, специалист по бизнес-развитию, исполнительный директор, профессор и т. д.</td>
    </tr>
    <tr>
      <td>ISTP</td>
      <td>Этот человек воспринимает мир через ощущения. По природе эмпат, но чаще сосредоточен на себе. Его умение объективно принимать решения и анализировать ситуацию указывает на технический склад ума. Он всегда соблюдает дедлайны, хотя иногда может поступить неожиданно</td>
      <td>Создатель: инженер, техник, строитель, инспектор, судебный эксперт, программист, разработчик ПО и т. д.</td>
    </tr>
    <tr>
      <td>ISFP</td>
      <td>Этот человек умеет находить радость в однообразии и рутинных делах. Прекрасно ладит с людьми, избегая конфликтов. Ему важно чувствовать свою значимость и оказывать помощь. Такой человек не стремится руководить или менять других, уважает их личные границы и ожидает того же в ответ. По натуре он приземленный практик, на которого всегда можно положиться</td>
      <td>Композитор: помощник по маркетингу, танцор, шеф-повар, офис-администратор, художник, дизайнер интерьеров, секретарь, медсестра и т. д.</td>
    </tr>
    <tr>
      <td>INFP</td>
      <td>Этот человек - чувствительный лирик, прекрасно разбирающийся в людях и легко вызывающий у них симпатию. Он обладает отличным чувством юмора и уделяет большое внимание своему внешнему виду. Стремится к самопознанию, гармонии с собой и старается быть полезным окружающим</td>
      <td>Целитель: писатель, дизайнер мультимедиа, менеджер по работе с клиентами, учитель для детей с особыми потребностями, тренер, редактор, модельер и т. д.</td>
    </tr>
    <tr>
      <td>INTP</td>
      <td>Этот человек - эрудит с философским складом ума. Он тщательно анализирует свои решения, стремясь к объективности и беспристрастности. Бурные проявления эмоций ему не свойственны. Однако большое количество данных и их изменчивость могут вызывать у него внутреннее напряжение</td>
      <td>Архитектор: технический писатель, веб-разработчик, аналитик информационной безопасности, исследователь, ученый, юрист и т. д.</td>
    </tr>
    <tr>
      <td>ESTP</td>
      <td>Этот человек всегда добивается успеха, невзирая на препятствия, которые лишь усиливают его целеустремленность. Он стремится к лидерским позициям и плохо переносит роль подчиненного. Обычно разрабатывает четкий план действий и неуклонно ему следует</td>
      <td>Промоутер: специалист по работе с клиентами, актер, личный тренер, бренд-амбассадор, менеджер, предприниматель, креативный директор, полицейский, маркетолог, производитель и т. д.</td>
    </tr>
    <tr>
      <td>ESFP</td>
      <td>Этот человек легко выявляет слабые стороны людей, что позволяет ему эффективно манипулировать и управлять. В общении он чаще всего руководствуется собственными интересами и предпочитает жить в настоящем. Часто не завершает начатое, стремясь к быстрым результатам. Однако при этом стремится поддерживать гармоничные отношения с окружающими</td>
      <td>Исполнитель: бортпроводник, артист, учитель, менеджер по связям с общественностью, торговый представитель, организатор мероприятий и т. д.</td>
    </tr>
    <tr>
      <td>ENFP</td>
      <td>Этот человек - творческая личность и фантазер, обладающий качествами, которые помогают ему успешно взаимодействовать с другими, быть открытым и общительным. Он активно участвует в различных мероприятиях, легко решает возникающие вопросы и демонстрирует гибкость</td>
      <td>Чемпион: медицинский работник, продюсер, продавец-консультант, специалист по обслуживанию клиентов, сценарист, ведущий на ТВ/радио и т. д.</td>
    </tr>
    <tr>
      <td>ENTP</td>
      <td>Этот человек - изобретательный, инициативный и гибкий. Он генератор идей и первопроходец, который не выносит рутины. Постоянное движение и интуитивное принятие решений всегда сопровождают его в работе</td>
      <td>Новатор: инженер, маркетолог, менеджер по социальным сетям, аналитик управления, руководитель цифрового маркетинга, бизнес-консультант, разработчик игр, менеджер по продажам и т. д.</td>
    </tr>
    <tr>
      <td>ESTJ</td>
      <td>Это трудолюбивый человек, который воспринимает мир таким, какой он есть. Он склонен тщательно планировать и доводить дела до конца. Заботится о своем ближайшем окружении, проявляет добродушие, но иногда может быть вспыльчивым, резким и упрямым</td>
      <td>Супервайзер: управляющий директор, менеджер отеля, финансовый сотрудник, судья, агент по недвижимости, генеральный директор, шеф-повар, менеджер по бизнес-развитию, телемаркетолог и т. д.</td>
    </tr>
    <tr>
      <td>ESFJ</td>
      <td>Этот человек умеет оказывать влияние на людей, проявляет заботу и готов жертвовать собой ради других. Он легко устанавливает контакт с любым человеком и способен направить ситуацию в нужное ему русло</td>
      <td>Поставщик: специалист по технической поддержке, менеджер по работе с клиентами, профессор колледжа, медицинский исследователь, бухгалтер, фотожурналист и т. д.</td>
    </tr>
    <tr>
      <td>ENFJ</td>
      <td>Этот человек отличается эмоциональностью и эмпатией. Его мимика выразительна, а речь — красноречива. Благодаря своей самоорганизованности, он успешно воплощает свои фантазии и идеи в жизнь. Он интуитивно понимает, какое решение следует принять в каждой конкретной ситуации</td>
      <td>Учитель: менеджер по связям с общественностью, менеджер по продажам, директор по управлению персоналом, арт-директор, консультант и т. д.</td>
    </tr>
    <tr>
      <td>ENTJ</td>
      <td>Этот человек легко увлекается, готов рисковать и полагается на интуицию. Без страха внедряет новые технологии и способен глубоко анализировать как себя, так и окружающий мир. Жизнь для него - это борьба, в которой он чувствует себя уверенно. Открыт для новых возможностей, но при этом нуждается в контроле</td>
      <td>Командир: руководитель строительства, администратор службы здравоохранения, финансовый бухгалтер, аудитор, юрист, директор школы, химический инженер, менеджер баз данных и т. д.</td>
    </tr>
  </table>
</div>

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим типам личности.

#### Ранжирование кандидатов по одному из шестнадцати типов личности по версии MBTI

In [8]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/493644095?token=EX7hFxNJhMoLumI&filename=df_mbti_correlation.csv'
df_correlation_coefficients = pd.read_csv(url)

df_correlation_coefficients.index.name = 'ID'
df_correlation_coefficients.index += 1
df_correlation_coefficients.index = df_correlation_coefficients.index.map(str)

df_correlation_coefficients

,Trait,EI,SN,TF,JP
ID,,,,,
1,Openness,0.09,-0.03,-0.14,-0.16
2,Conscientiousness,0.04,-0.04,0.20,0.14
3,Extraversion,0.20,-0.03,0.01,-0.07
4,Agreeableness,0.02,0.05,-0.35,0.03
5,Non-Neuroticism,0.08,0.00,0.16,0.00


In [9]:
_b5._professional_match(
    correlation_coefficients = df_correlation_coefficients,
    personality_type = "ENFJ",
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5._df_files_MBTI_job_match, name = 'MBTI_ranking_fi_en', out = True)

# Опционно
df = _b5.df_files_MBTI_job_match_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:6]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,EI,SN,TF,JP,MBTI,MBTI_Score,Match
Person ID,,,,,,,,,,,,,
5,be0DQawtVkE.002.mp4,0.681,0.566,0.554,0.647,0.642,0.259041,-0.027361,-0.100093,-0.049093,ENFP,0.289871,75.0
6,cLaZxEf1nE4.004.mp4,0.663,0.551,0.558,0.585,0.587,0.252010,-0.029419,-0.087981,-0.050501,ENFP,0.277057,75.0
10,_plk5k7PBEg.003.mp4,0.648,0.610,0.525,0.614,0.606,0.248447,-0.028874,-0.081294,-0.036454,ENFP,0.268961,75.0
8,JZNMxa3OKHY.000.mp4,0.606,0.524,0.531,0.594,0.580,0.239967,-0.025332,-0.090041,-0.042964,ENFP,0.266504,75.0
3,300gK3CnzW0.003.mp4,0.468,0.449,0.372,0.510,0.454,-0.160520,0.068617,-0.278880,0.053384,ISFJ,0.166132,50.0
1,2d6btbaNdfo.000.mp4,0.619,0.661,0.478,0.654,0.601,0.047788,0.002056,-0.092138,0.046539,ESFJ,0.139850,75.0
7,g24JGYuT74A.004.mp4,0.590,0.399,0.410,0.532,0.507,0.006447,0.037143,-0.271593,-0.105712,ESFP,0.139020,50.0
4,4vdJGgZpj4k.003.mp4,0.585,0.616,0.494,0.606,0.587,0.037527,0.002895,-0.081646,0.045425,ESFJ,0.123449,75.0
9,nvlqJbHk_Lc.003.mp4,0.511,0.465,0.391,0.444,0.439,-0.094752,-0.007199,-0.083317,-0.132767,INFP,0.045258,50.0


### `MuPTA` (ru)

In [10]:
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

corpus = 'mupta'
lang = 'ru'

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url, force_reload = False)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang=lang)
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['fe']['googledisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url, force_reload = False)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_translation_model = _b5.setup_translation_model() # только для русского языка
res_setup_translation_model = _b5.setup_bert_encoder(force_reload = False)
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['googledisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['text'][corpus]['nn']['googledisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url, force_reload = False)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['googledisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url, force_reload = False)

PATH_TO_DIR = './video_MuPTA/'
PATH_SAVE_VIDEO = './video_MuPTA/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса MuPTA
# URL: https://hci.nw.ru/en/pages/mupta-corpus
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '477995979?token=2cvyk7CS0mHx2MJ&filename=speaker_06_center_83.mov',
    '477995980?token=jGPtBPS69uzFU6Y&filename=speaker_01_center_83.mov',
    '477995967?token=zCaRbNB6ht5wMPq&filename=speaker_11_center_83.mov',
    '477995966?token=B1rbinDYRQKrI3T&filename=speaker_15_center_83.mov',
    '477995978?token=dEpVDtZg1EQiEQ9&filename=speaker_07_center_83.mov',
    '477995961?token=o1hVjw8G45q9L9Z&filename=speaker_19_center_83.mov',
    '477995964?token=5K220Aqf673VHPq&filename=speaker_23_center_83.mov',
    '477995965?token=v1LVD2KT1cU7Lpb&filename=speaker_24_center_83.mov',
    '477995962?token=tmaSGyyWLA6XCy9&filename=speaker_27_center_83.mov',
    '477995963?token=bTpo96qNDPcwGqb&filename=speaker_10_center_83.mov',
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mov'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_['mupta']['googledisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = lang)

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:24:03</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:24:04</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... GitHub\OCEANAI\docs\source\user_guide\notebooks\video_MuPTA\test\speaker_27_center_83.mov ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
Person ID,,,,,,
1,speaker_01_center_83.mov,0.765745,0.696637,0.656309,0.75986,0.494141
2,speaker_06_center_83.mov,0.686514,0.659488,0.611838,0.749739,0.420672
3,speaker_07_center_83.mov,0.671993,0.661216,0.571759,0.704542,0.381026
4,speaker_10_center_83.mov,0.69828,0.59893,0.571893,0.674907,0.35082
5,speaker_11_center_83.mov,0.718329,0.598986,0.573518,0.73201,0.379845
6,speaker_15_center_83.mov,0.670932,0.671055,0.602337,0.708656,0.399527
7,speaker_19_center_83.mov,0.767261,0.658167,0.653367,0.801366,0.463443
8,speaker_23_center_83.mov,0.699837,0.684907,0.616671,0.806437,0.447853
9,speaker_24_center_83.mov,0.710566,0.66299,0.610562,0.711242,0.413696


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:24:04</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.0706,0.0788,0.1328,0.1071,0.1002,0.0979
Accuracy,0.9294,0.9212,0.8672,0.8929,0.8998,0.9021


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:24:04</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.0979</span>, средняя точность: <span style="color:#1776D2">0.9021</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 322.244 сек. ---**</span>

True

<hr>

Для выполнения ранжирования кандидатов необходимо знать весовые коэффициенты, определяющие приоритетность персональных качеств личности в зависимости от профессии.

Предлагаются весовые коэффициенты для 5 профессий, вычисленные на основе научных статей:

1) Sajjad H. et al. Personality and Career Choices // African Journal of Business Management. - 2012. – Vol. 6 (6) – pp. 2255-2260.
2) Alkhelil A. H. The Relationship between Personality Traits and Career Choice: A Case Study of Secondary School Students // International Journal of Academic Research in Progressive Education and Development. – 2016. – Vol. 5(2). – pp. 2226-6348.
3) De Jong N. et al. Personality Traits and Career Role Enactment: Career Role Preferences as a Mediator // Frontiers in Psychology. – 2019. – Vol. 10. – pp. 1720.

Пользователь может установить свои весовые коэффициенты; сумма весов должна быть равна 100.

In [13]:
# Загрузка датафрейма с весовыми коэффициентами
url = 'https://download.sberdisk.ru/download/file/478675798?token=fF5fNZVpthQlEV0&filename=traits_priority_for_professions.csv'
traits_priority_for_professions = pd.read_csv(url)

traits_priority_for_professions.index.name = 'ID'
traits_priority_for_professions.index += 1
traits_priority_for_professions.index = traits_priority_for_professions.index.map(str)

traits_priority_for_professions

,Profession,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,Managers/executives,15,35,15,30,5
2,Entrepreneurship,30,30,5,5,30
3,Social/Non profit making professions,5,5,35,35,20
4,Public sector professions,15,50,15,15,5
5,"Scientists/researchers, and engineers",50,15,5,15,15


#### Ранжирование кандидатов на должность инженера-проектировщика

In [14]:
weights = traits_priority_for_professions.iloc[4].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'engineer_candidate_ranking_mupta_ru', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
10,speaker_27_center_83.mov,0.759,0.713,0.658,0.831,0.508,72.022
1,speaker_01_center_83.mov,0.766,0.697,0.656,0.760,0.494,70.828
7,speaker_19_center_83.mov,0.767,0.658,0.653,0.801,0.463,70.475
8,speaker_23_center_83.mov,0.700,0.685,0.617,0.806,0.448,67.163
9,speaker_24_center_83.mov,0.711,0.663,0.611,0.711,0.414,65.400
2,speaker_06_center_83.mov,0.687,0.659,0.612,0.750,0.421,64.833
5,speaker_11_center_83.mov,0.718,0.599,0.574,0.732,0.380,64.447
6,speaker_15_center_83.mov,0.671,0.671,0.602,0.709,0.400,63.247
3,speaker_07_center_83.mov,0.672,0.661,0.572,0.705,0.381,62.660


#### Ранжирование кандидатов на должность менеджера

In [15]:
weights = traits_priority_for_professions.iloc[0].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'executive_candidate_ranking_mupta_ru', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
10,speaker_27_center_83.mov,0.759,0.713,0.658,0.831,0.508,73.659
1,speaker_01_center_83.mov,0.766,0.697,0.656,0.760,0.494,70.980
7,speaker_19_center_83.mov,0.767,0.658,0.653,0.801,0.463,70.703
8,speaker_23_center_83.mov,0.700,0.685,0.617,0.806,0.448,70.152
2,speaker_06_center_83.mov,0.687,0.659,0.612,0.750,0.421,67.153
9,speaker_24_center_83.mov,0.711,0.663,0.611,0.711,0.414,66.427
6,speaker_15_center_83.mov,0.671,0.671,0.602,0.709,0.400,65.843
3,speaker_07_center_83.mov,0.672,0.661,0.572,0.705,0.381,64.840
5,speaker_11_center_83.mov,0.718,0.599,0.574,0.732,0.380,64.202


<hr>

Для ранжирования кандидатов по профессиональным навыкам необходимо задать по два коэффициента корреляции для каждого персонального качества личности человека и навыка, а также порога полярности качеств. Эти коэффициенты должны показывать, как измениться оценка качества человека если она больше или меньше заданного порога полярности качеств. 

В качестве примера предлагается использование коэффициентов корреляции между двумя людьми в четырьмя профессиональными навыками, представленных в статье:

1) Wehner C., de Grip A., Pfeifer H. Do recruiters select workers with different personality traits for different tasks? A discrete choice experiment // Labour Economics. - 2022. - vol. 78. - pp. 102186.

Представлены 4 профессиональных навыка: 

1) Analytical (Аналитические навыки). Умение эффективно решать новые задачи, требующие глубокого анализа. 
2) Interactive (Навыки межличностного общения). Умение убеждать и достигать компромиссов с заказчиками и коллегами.
4) Routine (Способность выполнять рутинную работу). Умение эффективно управлять рутинными задачами, соблюдая точность и внимание к деталям.
5) Non-Routine (Способность выполнять нестандартную работу). Умение реагировать и решать проблемы, не имеющие установленного порядка, проявляя адаптивность и креативные навыки в решении задач.

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим профессиональным навыкам.

#### Ранжирование кандидатов по профессиональным навыкам

In [16]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478678231?token=0qiZwliLtHWWYMv&filename=professional_skills.csv'
df_professional_skills = pd.read_csv(url)

df_professional_skills.index.name = 'ID'
df_professional_skills.index += 1
df_professional_skills.index = df_professional_skills.index.map(str)

df_professional_skills

,Trait,Score_level,Analytical,Interactive,Routine,Non-Routine
ID,,,,,,
1,Openness,high,0.082,0.348,0.571,0.510
2,Openness,low,0.196,0.152,0.148,0.218
3,Conscientiousness,high,0.994,1.333,1.507,1.258
4,Conscientiousness,low,0.241,0.188,0.191,0.267
5,Extraversion,high,0.169,-0.060,0.258,0.017
6,Extraversion,low,0.181,0.135,0.130,0.194
7,Agreeableness,high,1.239,0.964,1.400,1.191
8,Agreeableness,low,0.226,0.180,0.189,0.259
9,Non-Neuroticism,high,0.636,0.777,0.876,0.729


In [17]:
_b5._priority_skill_calculation(
    correlation_coefficients = df_professional_skills,
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_skill_, name = 'skill_candidate_ranking_mupta_ru', out = True)

# Опционно
df = _b5.df_files_priority_skill_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Analytical,Interactive,Routine,Non-Routine
Person ID,,,,,,,,,,
10,speaker_27_center_83.mov,0.759,0.713,0.658,0.831,0.508,0.447,0.474,0.657,0.531
8,speaker_23_center_83.mov,0.700,0.685,0.617,0.806,0.448,0.387,0.394,0.559,0.459
7,speaker_19_center_83.mov,0.767,0.658,0.653,0.801,0.463,0.383,0.390,0.559,0.459
1,speaker_01_center_83.mov,0.766,0.697,0.656,0.760,0.494,0.382,0.393,0.560,0.460
2,speaker_06_center_83.mov,0.687,0.659,0.612,0.750,0.421,0.366,0.374,0.533,0.437
9,speaker_24_center_83.mov,0.711,0.663,0.611,0.711,0.414,0.357,0.369,0.525,0.430
6,speaker_15_center_83.mov,0.671,0.671,0.602,0.709,0.400,0.357,0.368,0.522,0.427
3,speaker_07_center_83.mov,0.672,0.661,0.572,0.705,0.381,0.352,0.364,0.515,0.423
5,speaker_11_center_83.mov,0.718,0.599,0.574,0.732,0.380,0.347,0.356,0.510,0.418


<hr>

Для ранжирования кандидатов по одному из шестнадцати типов личности MBTI необходимо задать матрицу корреляции между персональными качествами личности человека и четырьмя диспозициями MBTI, установить порог полярности качеств и указать целевой тип личности MBTI.

В качестве примера предлагается использование коэффициентов корреляции, представленных в статье [1]. Описание типов личности MBTI и соотвествующие им успешные профессии представлены в статье [2].

1) Furnham A. The big five facets and the MBTI: The relationship between the 30 NEO-PI (R) Facets and the four Myers-Briggs Type Indicator (MBTI) scores // Psychology. - 2022. vol. 13(10). - pp. 1504-1516.
2) Tieger P.D., Barron B., Tieger K. Do what you are: Discover the perfect career for you through the secrets of personality type // Hachette UK. - 2024.

##### Типы личности MBTI основаны на четырех измерениях личности:

<div style="text-align: center;">
  <table style="width:100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Описание измерения</th>
      <th>Измерение</th>
    </tr>
    <tr>
      <td>Как мы взаимодействуем с миром и куда направляем свою энергию</td>
      <td>(E) Экстраверсия - Интроверсия (I)</td>
    </tr>
    <tr>
      <td>Вид информации, которую мы естественным образом замечаем</td>
      <td>(S) Сенсорика - Интуиция (N)</td>
    </tr>
    <tr>
      <td>Как мы принимаем решения</td>
      <td>(T) Логика - Чувства (F)</td>
    </tr>
    <tr>
      <td>Предпочитаем ли мы жить более структурированно (принимая решения) или более спонтанно (принимая информацию)</td>
      <td>(J) Оценка - Восприятие (P)</td>
    </tr>
  </table>
</div>

##### Типы личности MBTI и соотвествующие им успешные профессии:

<div style="text-align: center;">
  <table style="width: 100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Тип личности</th>
      <th>Описание</th>
      <th>Успешные профессии</th>
    </tr>
    <tr>
      <td>ISTJ</td>
      <td>Этот человек отличается ответственностью, строгостью и педантичностью. Он опирается на объективные факты и склонен к аналитическому мышлению. Приступает к задаче только тогда, когда уверен в своих возможностях и успехе</td>
      <td>Инспектор: бухгалтер, аудитор, бюджетный аналитик, финансовый менеджер, разработчик, системный аналитик, библиотекарь и т. д.</td>
    </tr>
    <tr>
      <td>ISFJ</td>
      <td>Этот человек склонен к самоанализу и анализу окружающих, легко распознает фальшь и предпочитает сохранять психологическую дистанцию. Он исполнителен, внимателен и готов помогать другим. Его силы и энергия исходят из внутренних ресурсов, и он всегда полагается на собственный опыт</td>
      <td>Защитник: медсестра, врач, ветеринар или ветеринарный ассистент, социальный работник, сельскохозяйственный или пищевой ученый, секретарь, водитель и т. д.</td>
    </tr>
    <tr>
      <td>INFJ</td>
      <td>О таких людях говорят: «ему можно доверять». Он отличается высокой чувствительностью, уделяет большое внимание межличностным отношениям, умеет давать ценные советы и помогает раскрывать потенциал других. Развитая интуиция не только генерирует множество идей, но и способствует самоорганизации</td>
      <td>Советник: психолог, специалист по управлению персоналом, офис-менеджер, специалист по обучению, графический дизайнер и т. д.</td>
    </tr>
    <tr>
      <td>INTJ</td>
      <td>Этот человек умеет выделять главное, говорит четко и по существу, придерживается практического подхода. Он стремится постоянно улучшать свою работу и всегда ищет способы сделать задачу еще лучше. Пустые разговоры ему не по душе, поэтому он избегает больших шумных компаний и с трудом заводит новые знакомства</td>
      <td>Мастермайнд: аниматор, архитектор, копирайтер, фотограф, тележурналист, видеомонтажер, специалист по бизнес-развитию, исполнительный директор, профессор и т. д.</td>
    </tr>
    <tr>
      <td>ISTP</td>
      <td>Этот человек воспринимает мир через ощущения. По природе эмпат, но чаще сосредоточен на себе. Его умение объективно принимать решения и анализировать ситуацию указывает на технический склад ума. Он всегда соблюдает дедлайны, хотя иногда может поступить неожиданно</td>
      <td>Создатель: инженер, техник, строитель, инспектор, судебный эксперт, программист, разработчик ПО и т. д.</td>
    </tr>
    <tr>
      <td>ISFP</td>
      <td>Этот человек умеет находить радость в однообразии и рутинных делах. Прекрасно ладит с людьми, избегая конфликтов. Ему важно чувствовать свою значимость и оказывать помощь. Такой человек не стремится руководить или менять других, уважает их личные границы и ожидает того же в ответ. По натуре он приземленный практик, на которого всегда можно положиться</td>
      <td>Композитор: помощник по маркетингу, танцор, шеф-повар, офис-администратор, художник, дизайнер интерьеров, секретарь, медсестра и т. д.</td>
    </tr>
    <tr>
      <td>INFP</td>
      <td>Этот человек - чувствительный лирик, прекрасно разбирающийся в людях и легко вызывающий у них симпатию. Он обладает отличным чувством юмора и уделяет большое внимание своему внешнему виду. Стремится к самопознанию, гармонии с собой и старается быть полезным окружающим</td>
      <td>Целитель: писатель, дизайнер мультимедиа, менеджер по работе с клиентами, учитель для детей с особыми потребностями, тренер, редактор, модельер и т. д.</td>
    </tr>
    <tr>
      <td>INTP</td>
      <td>Этот человек - эрудит с философским складом ума. Он тщательно анализирует свои решения, стремясь к объективности и беспристрастности. Бурные проявления эмоций ему не свойственны. Однако большое количество данных и их изменчивость могут вызывать у него внутреннее напряжение</td>
      <td>Архитектор: технический писатель, веб-разработчик, аналитик информационной безопасности, исследователь, ученый, юрист и т. д.</td>
    </tr>
    <tr>
      <td>ESTP</td>
      <td>Этот человек всегда добивается успеха, невзирая на препятствия, которые лишь усиливают его целеустремленность. Он стремится к лидерским позициям и плохо переносит роль подчиненного. Обычно разрабатывает четкий план действий и неуклонно ему следует</td>
      <td>Промоутер: специалист по работе с клиентами, актер, личный тренер, бренд-амбассадор, менеджер, предприниматель, креативный директор, полицейский, маркетолог, производитель и т. д.</td>
    </tr>
    <tr>
      <td>ESFP</td>
      <td>Этот человек легко выявляет слабые стороны людей, что позволяет ему эффективно манипулировать и управлять. В общении он чаще всего руководствуется собственными интересами и предпочитает жить в настоящем. Часто не завершает начатое, стремясь к быстрым результатам. Однако при этом стремится поддерживать гармоничные отношения с окружающими</td>
      <td>Исполнитель: бортпроводник, артист, учитель, менеджер по связям с общественностью, торговый представитель, организатор мероприятий и т. д.</td>
    </tr>
    <tr>
      <td>ENFP</td>
      <td>Этот человек - творческая личность и фантазер, обладающий качествами, которые помогают ему успешно взаимодействовать с другими, быть открытым и общительным. Он активно участвует в различных мероприятиях, легко решает возникающие вопросы и демонстрирует гибкость</td>
      <td>Чемпион: медицинский работник, продюсер, продавец-консультант, специалист по обслуживанию клиентов, сценарист, ведущий на ТВ/радио и т. д.</td>
    </tr>
    <tr>
      <td>ENTP</td>
      <td>Этот человек - изобретательный, инициативный и гибкий. Он генератор идей и первопроходец, который не выносит рутины. Постоянное движение и интуитивное принятие решений всегда сопровождают его в работе</td>
      <td>Новатор: инженер, маркетолог, менеджер по социальным сетям, аналитик управления, руководитель цифрового маркетинга, бизнес-консультант, разработчик игр, менеджер по продажам и т. д.</td>
    </tr>
    <tr>
      <td>ESTJ</td>
      <td>Это трудолюбивый человек, который воспринимает мир таким, какой он есть. Он склонен тщательно планировать и доводить дела до конца. Заботится о своем ближайшем окружении, проявляет добродушие, но иногда может быть вспыльчивым, резким и упрямым</td>
      <td>Супервайзер: управляющий директор, менеджер отеля, финансовый сотрудник, судья, агент по недвижимости, генеральный директор, шеф-повар, менеджер по бизнес-развитию, телемаркетолог и т. д.</td>
    </tr>
    <tr>
      <td>ESFJ</td>
      <td>Этот человек умеет оказывать влияние на людей, проявляет заботу и готов жертвовать собой ради других. Он легко устанавливает контакт с любым человеком и способен направить ситуацию в нужное ему русло</td>
      <td>Поставщик: специалист по технической поддержке, менеджер по работе с клиентами, профессор колледжа, медицинский исследователь, бухгалтер, фотожурналист и т. д.</td>
    </tr>
    <tr>
      <td>ENFJ</td>
      <td>Этот человек отличается эмоциональностью и эмпатией. Его мимика выразительна, а речь — красноречива. Благодаря своей самоорганизованности, он успешно воплощает свои фантазии и идеи в жизнь. Он интуитивно понимает, какое решение следует принять в каждой конкретной ситуации</td>
      <td>Учитель: менеджер по связям с общественностью, менеджер по продажам, директор по управлению персоналом, арт-директор, консультант и т. д.</td>
    </tr>
    <tr>
      <td>ENTJ</td>
      <td>Этот человек легко увлекается, готов рисковать и полагается на интуицию. Без страха внедряет новые технологии и способен глубоко анализировать как себя, так и окружающий мир. Жизнь для него - это борьба, в которой он чувствует себя уверенно. Открыт для новых возможностей, но при этом нуждается в контроле</td>
      <td>Командир: руководитель строительства, администратор службы здравоохранения, финансовый бухгалтер, аудитор, юрист, директор школы, химический инженер, менеджер баз данных и т. д.</td>
    </tr>
  </table>
</div>

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим типам личности.

In [18]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/493644095?token=EX7hFxNJhMoLumI&filename=df_mbti_correlation.csv'
df_correlation_coefficients = pd.read_csv(url)

df_correlation_coefficients.index.name = 'ID'
df_correlation_coefficients.index += 1
df_correlation_coefficients.index = df_correlation_coefficients.index.map(str)

df_correlation_coefficients

,Trait,EI,SN,TF,JP
ID,,,,,
1,Openness,0.09,-0.03,-0.14,-0.16
2,Conscientiousness,0.04,-0.04,0.20,0.14
3,Extraversion,0.20,-0.03,0.01,-0.07
4,Agreeableness,0.02,0.05,-0.35,0.03
5,Non-Neuroticism,0.08,0.00,0.16,0.00


In [19]:
_b5._professional_match(
    correlation_coefficients = df_correlation_coefficients,
    personality_type = "ENFJ",
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5._df_files_MBTI_job_match, name = 'MBTI_ranking_mupta_ru', out = True)

# Опционно
df = _b5.df_files_MBTI_job_match_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:6]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,EI,SN,TF,JP,MBTI,MBTI_Score,Match
Person ID,,,,,,,,,,,,,
7,speaker_19_center_83.mov,0.767,0.658,0.653,0.801,0.463,0.205006,-0.028877,-0.323879,-0.052313,ENFP,0.418321,75.0
1,speaker_01_center_83.mov,0.766,0.697,0.656,0.760,0.494,0.203710,-0.032534,-0.306328,-0.048136,ENFP,0.406929,75.0
8,speaker_23_center_83.mov,0.700,0.685,0.617,0.806,0.448,0.194016,-0.026570,-0.308738,-0.035061,ENFP,0.396993,75.0
2,speaker_06_center_83.mov,0.687,0.659,0.612,0.750,0.421,0.191874,-0.027843,-0.287812,-0.037850,ENFP,0.380647,75.0
5,speaker_11_center_83.mov,0.718,0.599,0.574,0.732,0.380,0.187565,-0.026114,-0.292012,-0.049261,ENFP,0.379269,75.0
9,speaker_24_center_83.mov,0.711,0.663,0.611,0.711,0.414,0.193712,-0.030591,-0.275902,-0.042274,ENFP,0.375154,75.0
6,speaker_15_center_83.mov,0.671,0.671,0.602,0.709,0.400,0.189904,-0.029607,-0.265650,-0.034305,ENFP,0.363871,75.0
10,speaker_27_center_83.mov,0.759,0.713,0.658,0.831,0.508,0.285739,-0.029510,-0.166680,-0.042916,ENFP,0.361447,75.0
4,speaker_10_center_83.mov,0.698,0.599,0.572,0.675,0.351,0.186613,-0.028317,-0.264603,-0.047660,ENFP,0.359650,75.0


### `MuPTA` (en)

In [21]:
import os
import pandas as pd

# Импорт модуля
from oceanai.modules.lab.build import Run

# Создание экземпляра класса
_b5 = Run()

corpus = 'fi'
lang = 'en'

# Настройка ядра
_b5.path_to_save_ = './models' # Директория для сохранения файла
_b5.chunk_size_ = 2000000      # Размер загрузки файла из сети за 1 шаг

# Формирование аудиомоделей
res_load_model_hc = _b5.load_audio_model_hc()
res_load_model_nn = _b5.load_audio_model_nn()

# Загрузка весов аудиомоделей
url = _b5.weights_for_big5_['audio'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_audio_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['audio'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_audio_model_weights_nn(url = url, force_reload = False)

# Формирование видеомоделей
res_load_model_hc = _b5.load_video_model_hc(lang=lang)
res_load_model_deep_fe = _b5.load_video_model_deep_fe()
res_load_model_nn = _b5.load_video_model_nn()

# Загрузка весов видеомоделей
url = _b5.weights_for_big5_['video'][corpus]['hc']['googledisk']
res_load_model_weights_hc = _b5.load_video_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['fe']['googledisk']
res_load_model_weights_deep_fe = _b5.load_video_model_weights_deep_fe(url = url, force_reload = False)

url = _b5.weights_for_big5_['video'][corpus]['nn']['googledisk']
res_load_model_weights_nn = _b5.load_video_model_weights_nn(url = url, force_reload = False)

# Загрузка словаря с экспертными признаками (текстовая модальность)
res_load_text_features = _b5.load_text_features()

# Формирование текстовых моделей 
res_setup_translation_model = _b5.setup_bert_encoder(force_reload = False)
res_load_text_model_hc_fi = _b5.load_text_model_hc(corpus=corpus)
res_load_text_model_nn_fi = _b5.load_text_model_nn(corpus=corpus)

# Загрузка весов текстовых моделей
url = _b5.weights_for_big5_['text'][corpus]['hc']['googledisk']
res_load_text_model_weights_hc_fi = _b5.load_text_model_weights_hc(url = url, force_reload = False)

url = _b5.weights_for_big5_['text'][corpus]['nn']['googledisk']
res_load_text_model_weights_nn_fi = _b5.load_text_model_weights_nn(url = url, force_reload = False)

# Формирование модели для мультимодального объединения информации
res_load_avt_model_b5 = _b5.load_avt_model_b5()

# Загрузка весов модели для мультимодального объединения информации
url = _b5.weights_for_big5_['avt'][corpus]['b5']['googledisk']
res_load_avt_model_weights_b5 = _b5.load_avt_model_weights_b5(url = url, force_reload = False)

PATH_TO_DIR = './video_MuPTA/'
PATH_SAVE_VIDEO = './video_MuPTA/test/'

_b5.path_to_save_ = PATH_SAVE_VIDEO

# Загрузка 10 тестовых аудиовидеозаписей из корпуса MuPTA
# URL: https://hci.nw.ru/en/pages/mupta-corpus
domain = 'https://download.sberdisk.ru/download/file/'
tets_name_files = [
    '477995979?token=2cvyk7CS0mHx2MJ&filename=speaker_06_center_83.mov',
    '477995980?token=jGPtBPS69uzFU6Y&filename=speaker_01_center_83.mov',
    '477995967?token=zCaRbNB6ht5wMPq&filename=speaker_11_center_83.mov',
    '477995966?token=B1rbinDYRQKrI3T&filename=speaker_15_center_83.mov',
    '477995978?token=dEpVDtZg1EQiEQ9&filename=speaker_07_center_83.mov',
    '477995961?token=o1hVjw8G45q9L9Z&filename=speaker_19_center_83.mov',
    '477995964?token=5K220Aqf673VHPq&filename=speaker_23_center_83.mov',
    '477995965?token=v1LVD2KT1cU7Lpb&filename=speaker_24_center_83.mov',
    '477995962?token=tmaSGyyWLA6XCy9&filename=speaker_27_center_83.mov',
    '477995963?token=bTpo96qNDPcwGqb&filename=speaker_10_center_83.mov',
]

for curr_files in tets_name_files:
    _b5.download_file_from_url(url = domain + curr_files, out = True)

# Получение прогнозов
_b5.path_to_dataset_ = PATH_TO_DIR # Директория набора данных
_b5.ext_ = ['.mov'] # Расширения искомых файлов

# Полный путь к файлу с верными предсказаниями для подсчета точности
url_accuracy = _b5.true_traits_['mupta']['googledisk']

_b5.get_avt_predictions(url_accuracy = url_accuracy, lang = lang)

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:40:54</span><span style="color:#666">]</span> <span style="color:#666">Извлечение признаков (экспертных и нейросетевых) из текста ...</span>** 

<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:40:55</span><span style="color:#666">]</span> <span style="color:#666">Получение прогнозов и вычисление точности (мультимодальное объединение) ...</span>**<p><span style="color:#666">&nbsp;&nbsp;&nbsp;&nbsp;10 из 10 (100.0%) ... GitHub\OCEANAI\docs\source\user_guide\notebooks\video_MuPTA\test\speaker_27_center_83.mov ...</span></p>

,Path,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
Person ID,,,,,,
1,speaker_01_center_83.mov,0.59561,0.542967,0.440668,0.589769,0.515306
2,speaker_06_center_83.mov,0.661347,0.673973,0.603208,0.64543,0.6431
3,speaker_07_center_83.mov,0.439868,0.465049,0.284547,0.422551,0.396058
4,speaker_10_center_83.mov,0.47715,0.502563,0.373686,0.441372,0.424637
5,speaker_11_center_83.mov,0.403292,0.344359,0.317304,0.422228,0.384346
6,speaker_15_center_83.mov,0.581837,0.562177,0.504623,0.602169,0.522254
7,speaker_19_center_83.mov,0.510444,0.448468,0.425599,0.451861,0.447891
8,speaker_23_center_83.mov,0.500526,0.541376,0.308529,0.441178,0.452412
9,speaker_24_center_83.mov,0.427677,0.511355,0.301078,0.434281,0.442301


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:40:55</span><span style="color:#666">]</span> <span style="color:#666">Точность по отдельным персональным качествам личности человека ...</span>** 

,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism,Mean
Metrics,,,,,,
MAE,0.1632,0.1621,0.176,0.2589,0.1122,0.1745
Accuracy,0.8368,0.8379,0.824,0.7411,0.8878,0.8255


<span style="color:#666">**[</span><span style="color:#1776D2">2024-10-10 17:40:55</span><span style="color:#666">]</span> <span style="color:#666">Средняя средних абсолютных ошибок: <span style="color:#1776D2">0.1745</span>, средняя точность: <span style="color:#1776D2">0.8255</span> ...</span>** 

<span style="color:#008001">**Лог файлы успешно сохранены ...**</span>

<span style="color:#666">**--- Время выполнения: 316.051 сек. ---**</span>

True

<hr>

Для выполнения ранжирования кандидатов необходимо знать весовые коэффициенты, определяющие приоритетность персональных качеств личности в зависимости от профессии.

Предлагаются весовые коэффициенты для 5 профессий, вычисленные на основе научных статей:

1) Sajjad H. et al. Personality and Career Choices // African Journal of Business Management. - 2012. – Vol. 6 (6) – pp. 2255-2260.
2) Alkhelil A. H. The Relationship between Personality Traits and Career Choice: A Case Study of Secondary School Students // International Journal of Academic Research in Progressive Education and Development. – 2016. – Vol. 5(2). – pp. 2226-6348.
3) De Jong N. et al. Personality Traits and Career Role Enactment: Career Role Preferences as a Mediator // Frontiers in Psychology. – 2019. – Vol. 10. – pp. 1720.

Пользователь может установить свои весовые коэффициенты; сумма весов должна быть равна 100.

In [22]:
# Загрузка датафрейма с весовыми коэффициентами
url = 'https://download.sberdisk.ru/download/file/478675798?token=fF5fNZVpthQlEV0&filename=traits_priority_for_professions.csv'
traits_priority_for_professions = pd.read_csv(url)

traits_priority_for_professions.index.name = 'ID'
traits_priority_for_professions.index += 1
traits_priority_for_professions.index = traits_priority_for_professions.index.map(str)

traits_priority_for_professions

,Profession,Openness,Conscientiousness,Extraversion,Agreeableness,Non-Neuroticism
ID,,,,,,
1,Managers/executives,15,35,15,30,5
2,Entrepreneurship,30,30,5,5,30
3,Social/Non profit making professions,5,5,35,35,20
4,Public sector professions,15,50,15,15,5
5,"Scientists/researchers, and engineers",50,15,5,15,15


#### Ранжирование кандидатов на должность инженера-проектировщика

In [23]:
weights = traits_priority_for_professions.iloc[4].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'engineer_candidate_ranking_mupta_en', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
2,speaker_06_center_83.mov,0.661,0.674,0.603,0.645,0.643,65.521
10,speaker_27_center_83.mov,0.566,0.659,0.434,0.591,0.579,57.934
6,speaker_15_center_83.mov,0.582,0.562,0.505,0.602,0.522,56.914
1,speaker_01_center_83.mov,0.596,0.543,0.441,0.590,0.515,56.704
8,speaker_23_center_83.mov,0.501,0.541,0.309,0.441,0.452,48.093
7,speaker_19_center_83.mov,0.510,0.448,0.426,0.452,0.448,47.873
4,speaker_10_center_83.mov,0.477,0.503,0.374,0.441,0.425,46.254
9,speaker_24_center_83.mov,0.428,0.511,0.301,0.434,0.442,43.708
3,speaker_07_center_83.mov,0.440,0.465,0.285,0.423,0.396,42.671


#### Ранжирование кандидатов на должность менеджера

In [24]:
weights = traits_priority_for_professions.iloc[0].values[1:]
weights = list(map(int, weights))

_b5._candidate_ranking(
    weigths_openness = weights[0], 
    weigths_conscientiousness = weights[1],
    weigths_extraversion = weights[2],
    weigths_agreeableness = weights[3], 
    weigths_non_neuroticism = weights[4],
    out = False
)

_b5._save_logs(df = _b5.df_files_ranking_, name = 'executive_candidate_ranking_mupta_en', out = True)

# Опционно
df = _b5.df_files_ranking_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Candidate score
Person ID,,,,,,,
2,speaker_06_center_83.mov,0.661,0.674,0.603,0.645,0.643,65.136
10,speaker_27_center_83.mov,0.566,0.659,0.434,0.591,0.579,58.710
6,speaker_15_center_83.mov,0.582,0.562,0.505,0.602,0.522,56.649
1,speaker_01_center_83.mov,0.596,0.543,0.441,0.590,0.515,54.818
8,speaker_23_center_83.mov,0.501,0.541,0.309,0.441,0.452,46.581
4,speaker_10_center_83.mov,0.477,0.503,0.374,0.441,0.425,45.717
7,speaker_19_center_83.mov,0.510,0.448,0.426,0.452,0.448,45.532
9,speaker_24_center_83.mov,0.428,0.511,0.301,0.434,0.442,44.069
3,speaker_07_center_83.mov,0.440,0.465,0.285,0.423,0.396,41.800


<hr>

Для ранжирования кандидатов по профессиональным навыкам необходимо задать по два коэффициента корреляции для каждого персонального качества личности человека и навыка, а также порога полярности качеств. Эти коэффициенты должны показывать, как измениться оценка качества человека если она больше или меньше заданного порога полярности качеств. 

В качестве примера предлагается использование коэффициентов корреляции между двумя людьми в четырьмя профессиональными навыками, представленных в статье:

1) Wehner C., de Grip A., Pfeifer H. Do recruiters select workers with different personality traits for different tasks? A discrete choice experiment // Labour Economics. - 2022. - vol. 78. - pp. 102186.

Представлены 4 профессиональных навыка: 

1) Analytical (Аналитические навыки). Умение эффективно решать новые задачи, требующие глубокого анализа. 
2) Interactive (Навыки межличностного общения). Умение убеждать и достигать компромиссов с заказчиками и коллегами.
4) Routine (Способность выполнять рутинную работу). Умение эффективно управлять рутинными задачами, соблюдая точность и внимание к деталям.
5) Non-Routine (Способность выполнять нестандартную работу). Умение реагировать и решать проблемы, не имеющие установленного порядка, проявляя адаптивность и креативные навыки в решении задач.

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим профессиональным навыкам.

#### Ранжирование кандидатов по профессиональным навыкам

In [25]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/478678231?token=0qiZwliLtHWWYMv&filename=professional_skills.csv'
df_professional_skills = pd.read_csv(url)

df_professional_skills.index.name = 'ID'
df_professional_skills.index += 1
df_professional_skills.index = df_professional_skills.index.map(str)

df_professional_skills

,Trait,Score_level,Analytical,Interactive,Routine,Non-Routine
ID,,,,,,
1,Openness,high,0.082,0.348,0.571,0.510
2,Openness,low,0.196,0.152,0.148,0.218
3,Conscientiousness,high,0.994,1.333,1.507,1.258
4,Conscientiousness,low,0.241,0.188,0.191,0.267
5,Extraversion,high,0.169,-0.060,0.258,0.017
6,Extraversion,low,0.181,0.135,0.130,0.194
7,Agreeableness,high,1.239,0.964,1.400,1.191
8,Agreeableness,low,0.226,0.180,0.189,0.259
9,Non-Neuroticism,high,0.636,0.777,0.876,0.729


In [26]:
_b5._priority_skill_calculation(
    correlation_coefficients = df_professional_skills,
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5.df_files_priority_skill_, name = 'skill_candidate_ranking_mupta_en', out = True)

# Опционно
df = _b5.df_files_priority_skill_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,Analytical,Interactive,Routine,Non-Routine
Person ID,,,,,,,,,,
2,speaker_06_center_83.mov,0.661,0.674,0.603,0.645,0.643,0.407,0.443,0.603,0.487
10,speaker_27_center_83.mov,0.566,0.659,0.434,0.591,0.579,0.376,0.431,0.542,0.466
6,speaker_15_center_83.mov,0.582,0.562,0.505,0.602,0.522,0.354,0.382,0.522,0.422
1,speaker_01_center_83.mov,0.596,0.543,0.441,0.590,0.515,0.345,0.392,0.499,0.430
4,speaker_10_center_83.mov,0.477,0.503,0.374,0.441,0.425,0.170,0.188,0.206,0.205
9,speaker_24_center_83.mov,0.428,0.511,0.301,0.434,0.442,0.167,0.187,0.206,0.203
8,speaker_23_center_83.mov,0.501,0.541,0.309,0.441,0.452,0.166,0.218,0.260,0.244
3,speaker_07_center_83.mov,0.440,0.465,0.285,0.423,0.396,0.085,0.066,0.067,0.096
7,speaker_19_center_83.mov,0.510,0.448,0.426,0.452,0.448,0.084,0.094,0.118,0.137


<hr>

Для ранжирования кандидатов по одному из шестнадцати типов личности MBTI необходимо задать матрицу корреляции между персональными качествами личности человека и четырьмя диспозициями MBTI, установить порог полярности качеств и указать целевой тип личности MBTI.

В качестве примера предлагается использование коэффициентов корреляции, представленных в статье [1]. Описание типов личности MBTI и соотвествующие им успешные профессии представлены в статье [2].

1) Furnham A. The big five facets and the MBTI: The relationship between the 30 NEO-PI (R) Facets and the four Myers-Briggs Type Indicator (MBTI) scores // Psychology. - 2022. vol. 13(10). - pp. 1504-1516.
2) Tieger P.D., Barron B., Tieger K. Do what you are: Discover the perfect career for you through the secrets of personality type // Hachette UK. - 2024.

##### Типы личности MBTI основаны на четырех измерениях личности:

<div style="text-align: center;">
  <table style="width:100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Описание измерения</th>
      <th>Измерение</th>
    </tr>
    <tr>
      <td>Как мы взаимодействуем с миром и куда направляем свою энергию</td>
      <td>(E) Экстраверсия - Интроверсия (I)</td>
    </tr>
    <tr>
      <td>Вид информации, которую мы естественным образом замечаем</td>
      <td>(S) Сенсорика - Интуиция (N)</td>
    </tr>
    <tr>
      <td>Как мы принимаем решения</td>
      <td>(T) Логика - Чувства (F)</td>
    </tr>
    <tr>
      <td>Предпочитаем ли мы жить более структурированно (принимая решения) или более спонтанно (принимая информацию)</td>
      <td>(J) Оценка - Восприятие (P)</td>
    </tr>
  </table>
</div>

##### Типы личности MBTI и соотвествующие им успешные профессии:

<div style="text-align: center;">
  <table style="width: 100%; margin: 0 auto; font-size: 0.6em;">
    <tr>
      <th>Тип личности</th>
      <th>Описание</th>
      <th>Успешные профессии</th>
    </tr>
    <tr>
      <td>ISTJ</td>
      <td>Этот человек отличается ответственностью, строгостью и педантичностью. Он опирается на объективные факты и склонен к аналитическому мышлению. Приступает к задаче только тогда, когда уверен в своих возможностях и успехе</td>
      <td>Инспектор: бухгалтер, аудитор, бюджетный аналитик, финансовый менеджер, разработчик, системный аналитик, библиотекарь и т. д.</td>
    </tr>
    <tr>
      <td>ISFJ</td>
      <td>Этот человек склонен к самоанализу и анализу окружающих, легко распознает фальшь и предпочитает сохранять психологическую дистанцию. Он исполнителен, внимателен и готов помогать другим. Его силы и энергия исходят из внутренних ресурсов, и он всегда полагается на собственный опыт</td>
      <td>Защитник: медсестра, врач, ветеринар или ветеринарный ассистент, социальный работник, сельскохозяйственный или пищевой ученый, секретарь, водитель и т. д.</td>
    </tr>
    <tr>
      <td>INFJ</td>
      <td>О таких людях говорят: «ему можно доверять». Он отличается высокой чувствительностью, уделяет большое внимание межличностным отношениям, умеет давать ценные советы и помогает раскрывать потенциал других. Развитая интуиция не только генерирует множество идей, но и способствует самоорганизации</td>
      <td>Советник: психолог, специалист по управлению персоналом, офис-менеджер, специалист по обучению, графический дизайнер и т. д.</td>
    </tr>
    <tr>
      <td>INTJ</td>
      <td>Этот человек умеет выделять главное, говорит четко и по существу, придерживается практического подхода. Он стремится постоянно улучшать свою работу и всегда ищет способы сделать задачу еще лучше. Пустые разговоры ему не по душе, поэтому он избегает больших шумных компаний и с трудом заводит новые знакомства</td>
      <td>Мастермайнд: аниматор, архитектор, копирайтер, фотограф, тележурналист, видеомонтажер, специалист по бизнес-развитию, исполнительный директор, профессор и т. д.</td>
    </tr>
    <tr>
      <td>ISTP</td>
      <td>Этот человек воспринимает мир через ощущения. По природе эмпат, но чаще сосредоточен на себе. Его умение объективно принимать решения и анализировать ситуацию указывает на технический склад ума. Он всегда соблюдает дедлайны, хотя иногда может поступить неожиданно</td>
      <td>Создатель: инженер, техник, строитель, инспектор, судебный эксперт, программист, разработчик ПО и т. д.</td>
    </tr>
    <tr>
      <td>ISFP</td>
      <td>Этот человек умеет находить радость в однообразии и рутинных делах. Прекрасно ладит с людьми, избегая конфликтов. Ему важно чувствовать свою значимость и оказывать помощь. Такой человек не стремится руководить или менять других, уважает их личные границы и ожидает того же в ответ. По натуре он приземленный практик, на которого всегда можно положиться</td>
      <td>Композитор: помощник по маркетингу, танцор, шеф-повар, офис-администратор, художник, дизайнер интерьеров, секретарь, медсестра и т. д.</td>
    </tr>
    <tr>
      <td>INFP</td>
      <td>Этот человек - чувствительный лирик, прекрасно разбирающийся в людях и легко вызывающий у них симпатию. Он обладает отличным чувством юмора и уделяет большое внимание своему внешнему виду. Стремится к самопознанию, гармонии с собой и старается быть полезным окружающим</td>
      <td>Целитель: писатель, дизайнер мультимедиа, менеджер по работе с клиентами, учитель для детей с особыми потребностями, тренер, редактор, модельер и т. д.</td>
    </tr>
    <tr>
      <td>INTP</td>
      <td>Этот человек - эрудит с философским складом ума. Он тщательно анализирует свои решения, стремясь к объективности и беспристрастности. Бурные проявления эмоций ему не свойственны. Однако большое количество данных и их изменчивость могут вызывать у него внутреннее напряжение</td>
      <td>Архитектор: технический писатель, веб-разработчик, аналитик информационной безопасности, исследователь, ученый, юрист и т. д.</td>
    </tr>
    <tr>
      <td>ESTP</td>
      <td>Этот человек всегда добивается успеха, невзирая на препятствия, которые лишь усиливают его целеустремленность. Он стремится к лидерским позициям и плохо переносит роль подчиненного. Обычно разрабатывает четкий план действий и неуклонно ему следует</td>
      <td>Промоутер: специалист по работе с клиентами, актер, личный тренер, бренд-амбассадор, менеджер, предприниматель, креативный директор, полицейский, маркетолог, производитель и т. д.</td>
    </tr>
    <tr>
      <td>ESFP</td>
      <td>Этот человек легко выявляет слабые стороны людей, что позволяет ему эффективно манипулировать и управлять. В общении он чаще всего руководствуется собственными интересами и предпочитает жить в настоящем. Часто не завершает начатое, стремясь к быстрым результатам. Однако при этом стремится поддерживать гармоничные отношения с окружающими</td>
      <td>Исполнитель: бортпроводник, артист, учитель, менеджер по связям с общественностью, торговый представитель, организатор мероприятий и т. д.</td>
    </tr>
    <tr>
      <td>ENFP</td>
      <td>Этот человек - творческая личность и фантазер, обладающий качествами, которые помогают ему успешно взаимодействовать с другими, быть открытым и общительным. Он активно участвует в различных мероприятиях, легко решает возникающие вопросы и демонстрирует гибкость</td>
      <td>Чемпион: медицинский работник, продюсер, продавец-консультант, специалист по обслуживанию клиентов, сценарист, ведущий на ТВ/радио и т. д.</td>
    </tr>
    <tr>
      <td>ENTP</td>
      <td>Этот человек - изобретательный, инициативный и гибкий. Он генератор идей и первопроходец, который не выносит рутины. Постоянное движение и интуитивное принятие решений всегда сопровождают его в работе</td>
      <td>Новатор: инженер, маркетолог, менеджер по социальным сетям, аналитик управления, руководитель цифрового маркетинга, бизнес-консультант, разработчик игр, менеджер по продажам и т. д.</td>
    </tr>
    <tr>
      <td>ESTJ</td>
      <td>Это трудолюбивый человек, который воспринимает мир таким, какой он есть. Он склонен тщательно планировать и доводить дела до конца. Заботится о своем ближайшем окружении, проявляет добродушие, но иногда может быть вспыльчивым, резким и упрямым</td>
      <td>Супервайзер: управляющий директор, менеджер отеля, финансовый сотрудник, судья, агент по недвижимости, генеральный директор, шеф-повар, менеджер по бизнес-развитию, телемаркетолог и т. д.</td>
    </tr>
    <tr>
      <td>ESFJ</td>
      <td>Этот человек умеет оказывать влияние на людей, проявляет заботу и готов жертвовать собой ради других. Он легко устанавливает контакт с любым человеком и способен направить ситуацию в нужное ему русло</td>
      <td>Поставщик: специалист по технической поддержке, менеджер по работе с клиентами, профессор колледжа, медицинский исследователь, бухгалтер, фотожурналист и т. д.</td>
    </tr>
    <tr>
      <td>ENFJ</td>
      <td>Этот человек отличается эмоциональностью и эмпатией. Его мимика выразительна, а речь — красноречива. Благодаря своей самоорганизованности, он успешно воплощает свои фантазии и идеи в жизнь. Он интуитивно понимает, какое решение следует принять в каждой конкретной ситуации</td>
      <td>Учитель: менеджер по связям с общественностью, менеджер по продажам, директор по управлению персоналом, арт-директор, консультант и т. д.</td>
    </tr>
    <tr>
      <td>ENTJ</td>
      <td>Этот человек легко увлекается, готов рисковать и полагается на интуицию. Без страха внедряет новые технологии и способен глубоко анализировать как себя, так и окружающий мир. Жизнь для него - это борьба, в которой он чувствует себя уверенно. Открыт для новых возможностей, но при этом нуждается в контроле</td>
      <td>Командир: руководитель строительства, администратор службы здравоохранения, финансовый бухгалтер, аудитор, юрист, директор школы, химический инженер, менеджер баз данных и т. д.</td>
    </tr>
  </table>
</div>

Пользователь может установить свои коэффициенты корреляции и ранжировать кандидатов по другим типам личности.

In [27]:
# Загрузка датафрейма с коэффициентами корреляции
url = 'https://download.sberdisk.ru/download/file/493644095?token=EX7hFxNJhMoLumI&filename=df_mbti_correlation.csv'
df_correlation_coefficients = pd.read_csv(url)

df_correlation_coefficients.index.name = 'ID'
df_correlation_coefficients.index += 1
df_correlation_coefficients.index = df_correlation_coefficients.index.map(str)

df_correlation_coefficients

,Trait,EI,SN,TF,JP
ID,,,,,
1,Openness,0.09,-0.03,-0.14,-0.16
2,Conscientiousness,0.04,-0.04,0.20,0.14
3,Extraversion,0.20,-0.03,0.01,-0.07
4,Agreeableness,0.02,0.05,-0.35,0.03
5,Non-Neuroticism,0.08,0.00,0.16,0.00


In [30]:
_b5._professional_match(
    correlation_coefficients = df_correlation_coefficients,
    personality_type = "ENFJ",
    threshold = 0.5,
    out = True
)

_b5._save_logs(df = _b5._df_files_MBTI_job_match, name = 'MBTI_ranking_mupta_en', out = True)

# Опционно
df = _b5.df_files_MBTI_job_match_.rename(columns = {'Openness':'OPE', 'Conscientiousness':'CON', 'Extraversion': 'EXT', 'Agreeableness': 'AGR', 'Non-Neuroticism': 'NNEU'})
columns_to_round = df.columns[1:6]
df[columns_to_round] = df[columns_to_round].apply(lambda x: [round(i, 3) for i in x])
df

,Path,OPE,CON,EXT,AGR,NNEU,EI,SN,TF,JP,MBTI,MBTI_Score,Match
Person ID,,,,,,,,,,,,,
2,speaker_06_center_83.mov,0.661,0.674,0.603,0.645,0.643,0.271478,-0.032624,-0.074766,-0.034321,ENFP,0.284152,75.0
6,speaker_15_center_83.mov,0.582,0.562,0.505,0.602,0.522,0.229601,-0.024972,-0.091174,-0.031648,ENFP,0.259311,75.0
10,speaker_27_center_83.mov,0.566,0.659,0.434,0.591,0.579,0.048690,-0.000776,-0.066064,0.049778,ENFJ,0.165309,100.0
1,speaker_01_center_83.mov,0.596,0.543,0.441,0.590,0.515,0.040210,0.003122,-0.103169,0.029258,ESFJ,0.129477,75.0
4,speaker_10_center_83.mov,0.477,0.503,0.374,0.441,0.425,-0.140376,-0.016646,0.250115,0.159620,INTJ,0.088133,50.0
9,speaker_24_center_83.mov,0.428,0.511,0.301,0.434,0.442,-0.122322,-0.020306,0.240365,0.148065,INTJ,0.084185,50.0
7,speaker_19_center_83.mov,0.510,0.448,0.426,0.452,0.448,-0.101987,-0.007200,-0.078923,-0.128220,INFP,0.043061,50.0
8,speaker_23_center_83.mov,0.501,0.541,0.309,0.441,0.452,-0.040020,-0.049474,0.117143,0.004070,INTJ,0.026772,50.0
5,speaker_11_center_83.mov,0.403,0.344,0.317,0.422,0.384,-0.152724,0.014281,0.070700,0.025861,ISTJ,0.006465,25.0
